# INSERTION DONNEES FINALES

###### 

In [3]:
# Import des librairies

import os
import pandas as pd
import psycopg2
import csv
import reprlib
import numpy
import json

######

---------------

## Insertion des nouvelles données

######

### Insertion origines

###### 

In [4]:
# Prod
# liste des origines pour insertions

import csv
# je crée une liste regroupant ma liste de pays
liste = []
with open('rumy_final.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        # gestion des doublons
        if row[2] not in liste:
            liste.append(row[2])
liste = liste[1:]
# print(liste)
# print(len(liste))

In [5]:
# connexion à la bdd collection
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection5"
)
cur = conn.cursor()

for pays in liste:
    cur.execute("SELECT origine.nom FROM origine WHERE origine.nom = (%s) ", (pays,))
    verif = cur.fetchone()
    if not verif:
        cur.execute("INSERT INTO origine (nom) VALUES(%s)", (pays,))

conn.commit()
print('insertion terminée')
conn.close()

insertion terminée


######

----------------

### Insertion producteurs

######

In [6]:
# je crée une liste regroupant les pays et les producteurs
liste_co = []
liste_producteur = []
liste_final = []
with open('rumy_final.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if row[1:3] not in liste_co:
            liste_co.append((row[1:3]))
liste_co = liste_co[1:]

In [7]:
# prod
# fonction d'insertion dans la table producteur
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection5"
)
cur = conn.cursor()
for producteur, pays in liste_co:
#     dictionnaire.items()
#     print(producteur)
#     print(pays)
    cur.execute(f"""SELECT origine.code_origine 
        FROM origine
        WHERE origine.nom = '{pays}';
        """)
    
    origine = cur.fetchone()[0]
#     print(origine)
    #pays = str(pro[0])
#     print(pays)

    cur.execute(f'''INSERT INTO producteur (nom_producteur, code_origine) VALUES (%s, %s)''', (producteur, origine))
    
cur.close()
conn.commit()
print('insertion terminée')
conn.close()

insertion terminée


###### 

___________

### Insertion produits

###### 

In [8]:
# insertion fichier excel
csv_file = 'rumy_final.csv'
inventaire = pd.read_csv(csv_file)
inventaire.head()

,nom,producteur,origine,volume,degre,type
0,Eminente Rhum Vieux 7 ans Reserva,Eminente,Cuba,70.0,41.3,Vieux
1,La Progresiva Rhum Vieux,La progresiva,Cuba,NaN,41.0,Vieux
2,Black Tears Rhum Épice,Black tears,Cuba,NaN,40.0,NaN
3,Conde de Liqueur Elixir,Conde,Cuba,70.0,32.0,Spiced
4,Legendario Rhum Ambré Ron Dorado,Legendario,Cuba,70.0,38.0,NaN


In [9]:
# cchangement des colonnes
inventaire_produit =  inventaire[['nom', 'volume', 'degre', 'producteur', 'type']]

In [10]:
liste_inventaire = [list(x) for x in inventaire_produit.to_numpy()]
print(liste_inventaire[0:10])

[['Eminente Rhum Vieux  7 ans Reserva   ', 70.0, 41.3, 'Eminente', 'Vieux'], ['La Progresiva Rhum Vieux   ', nan, 41.0, 'La progresiva', 'Vieux'], ['Black Tears Rhum Épice  ', nan, 40.0, 'Black tears', nan], ['Conde de  Liqueur Elixir   ', 70.0, 32.0, 'Conde', 'Spiced'], ['Legendario Rhum Ambré Ron Dorado   ', 70.0, 38.0, 'Legendario', nan], ['Havana Club Rhum Vieux Union coffret   ', 70.0, 40.0, 'Havana club', 'Coffret'], ['Conde de  Rhum Vieux 7 ans   ', 70.0, 38.0, 'Conde de ', 'Vieux'], ['Bristol Rhum Vieux Fine n 2003 9 ans   ', 70.0, 43.0, 'Bristol', 'Vieux'], ['Caney Rhum Blanc 3 ans Carta Blanca   ', 70.0, 38.0, 'Caney', 'Blanc'], ['Caney Rhum Vieux 7 ans Anejo Centuria   ', 70.0, 38.0, 'Caney', 'Vieux']]


In [11]:
nom = liste_inventaire[0][0]
volume = liste_inventaire[0][1]
degre = liste_inventaire[0][2]
producteur = liste_inventaire[0][3]
type_r = liste_inventaire[0][4]


print(nom)
print((volume))
print(degre)
print(producteur)
print(type_r)


Eminente Rhum Vieux  7 ans Reserva   
70.0
41.3
Eminente
Vieux


In [12]:
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection5"
)
cur = conn.cursor()

for nom, volume, degre, producteur, type_r in liste_inventaire:
    
#     print(nom)
#     print(volume)
#     print(degre)
#     print(producteur)
#     print(type_r)

    cur.execute("""SELECT producteur.code_producteur 
        FROM producteur
        WHERE producteur.nom_producteur = %s;
        """, (producteur, ))
    
    code = cur.fetchone()[0]

    if volume > 0 and degre > 0:
        cur.execute("INSERT INTO produit (nom_produit, volume, degre, type_rhum, code_producteur) VALUES (%s, %s, %s, %s, %s)", (nom, volume, degre, type_r, code))
    elif volume < 0:
        cur.execute(f"INSERT INTO produit (nom_produit, degre, type_rhum, code_producteur) VALUES (%s, %s, %s, %s)", (nom, degre, type_r, code))
    elif degre < 0:
        cur.execute(f"INSERT INTO produit (nom_produit, volume, type_rhum, code_producteur) VALUES (%s, %s, %s, %s)", (nom, volume, type_r, code))

    else:
         cur.execute(f"INSERT INTO produit (nom_produit, type_rhum, code_producteur) VALUES (%s, %s, %s)", (nom, type_r, code))
#         print(type(cur))   
cur.close()
conn.commit()
print('insertion produit terminée')
conn.close()

insertion produit terminée
